In [1]:
import pandas as pd
import gensim
import numpy as np
from gensim.models import word2vec
import preprocessing
from gensim.models import Phrases

<h1>Import preprocessed data</h1>

Data from file created in get_stack_data 

In [2]:
data = pd.read_csv('DB/Preprocessed_data.csv')

data

,id,title,tags,score,answers_processed,questions_processed,processed_title,questions
0,742,Class views in Django,django|views|oop,0.006875,needed use class based views wanted able use f...,view points function problem want change bit f...,class views django,"view points to a function, which can be a pr..."
1,2933,Create a directly-executable cross-platform GU...,user-interface|deployment|tkinter|release-mana...,0.039978,pysimplegui wraps tkinter works python also ru...,python works multiple platforms used desktop w...,create directly executable cross platform gui ...,Python works on multiple platforms and can be...
2,19339,Transpose/Unzip Function (inverse of zip)?,list|matrix|transpose,0.093733,since returns tuples trick seems clever useful...,list item tuples like convert lists first cont...,transpose unzip function inverse zip,I have a list of 2-item tuples and I'd like t...
3,21961,Date/time conversion using time.mktime seems w...,datetime,0.000650,local time fancy time tuple incidentally seem ...,return number seconds since epoch since giving...,date time conversion using time mktime seems w...,should return the number of seconds sinc...
4,36139,How to sort a list of strings?,string|sorting,0.067745,proper way sort strings previous example work ...,best way creating alphabetically sorted list p...,sort list strings,What is the best way of creating an alphabeti...
...,...,...,...,...,...,...,...,...
134908,69059605,How to check if a column of DataFrame contain ...,pandas|dataframe|types,-0.000338,try even outputs create subset dataframe check...,example got data frame gt gt question filter e...,check column dataframe contain float type,"For example, I've got this data frame. ..."
134909,69059907,How to increase/change a value in a dataset wi...,pandas|for-loop,-0.000536,create boolean auxiallry column multiply value...,want create loop iterates data probability end...,increase change value dataset condition python,What I want to do is: create a for loop that ...
134910,69059948,Check if elements in numpy array are between c...,numpy,-0.000338,direct way add axis broadcast think broadcasti...,numpy array arbitrary size numpy array exactly...,check elements numpy array columns column nump...,I have a 1-D numpy array with arbitrary size...
134911,69060114,Pytorch geometric: how to explain the input in...,machine-learning|graph|pytorch|data-science,-0.000536,okay think got understanding output dimensions...,reading pytorch geometric documentation page c...,pytorch geometric explain input code snippet,I am reading PyTorch geometric documentation ...


From the dataframe tokenize sentence from the titles, questions and anwers.
First create lists from column and than tokenize it and store that in documents.

In [3]:
titles = data.processed_title.tolist()
questions = data.questions_processed.tolist()
answers = data.answers_processed.tolist()

docs1 = [_text.split() for _text in np.array(titles)]
docs2 = [_text.split() for _text in np.array(questions)]
docs3 = [_text.split() for _text in np.array(answers)]

documents = docs1 + docs2 + docs3

<h1>Train Word Embeddings</h1>

Generate word to vec model.

Parameter:
- Vector size : dimensionality of the word vectors.
- Window :  maximum distance between the current and predicted word within a sentence.
- Min Count : ignores all words with total frequency lower than this.
- Workers : put here the number of cores of your cpu to speed up model creation

Build vocabulary and then train and save to file the model pre created with documents.

In [4]:
VECTOR_SIZE = 300
WINDOW_SIZE = 7
EPOCH = 32
MIN_COUNT = 10

In [5]:
word_2_vec_model = gensim.models.word2vec.Word2Vec(vector_size = VECTOR_SIZE, 
                                                   window = WINDOW_SIZE, min_count = MIN_COUNT, workers = 4 )

word_2_vec_model.build_vocab(documents)
words = list(word_2_vec_model.wv.index_to_key)
sizeVocab = len(words)

words
print('Size vocab:', sizeVocab)

word_2_vec_model.train(documents, total_examples = len(documents), epochs = EPOCH)

word_2_vec_model.save('DB/word2vec_trained.bin')

Size vocab: 16346


<h2>Sanity checks on train word embeddings</h2>

In [6]:
word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')


print('Terms most similar to "dataframe"')
print(word_2_vec_model.wv.most_similar("dataframe"))
print('-'*100)

print('Terms most similar to "pandas"')
print(word_2_vec_model.wv.most_similar("pandas"))
print('-'*100)

print('Terms most similar to "indexing"')
print(word_2_vec_model.wv.most_similar("indexing"))

Terms most similar to "dataframe"
[('df', 0.8094456791877747), ('column', 0.7701472043991089), ('columns', 0.7136840224266052), ('dataframes', 0.7055155634880066), ('rows', 0.6264829039573669), ('row', 0.6093869805335999), ('series', 0.6053846478462219), ('frame', 0.5846728682518005), ('multiindex', 0.5692790746688843), ('groupby', 0.5318722724914551)]
----------------------------------------------------------------------------------------------------
Terms most similar to "pandas"
[('panda', 0.6627941727638245), ('pd', 0.5611469745635986), ('dataframe', 0.5197407007217407), ('pyspark', 0.4814746677875519), ('columns', 0.44698476791381836), ('dask', 0.432192325592041), ('dataframes', 0.4259294271469116), ('column', 0.4185863733291626), ('groupby', 0.4151650071144104), ('df', 0.4103936553001404)]
----------------------------------------------------------------------------------------------------
Terms most similar to "indexing"
[('slicing', 0.6648306846618652), ('slice', 0.5238271355628